In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
%matplotlib inline
import pickle

In [2]:
dense_text=True
SEED=411 
kf = KFold(5, shuffle=True, random_state=SEED)

In [3]:
train = pickle.load(open('train_selected_features', 'rb'))
test = pickle.load(open('test_selected_features', 'rb'))
train.shape, test.shape

((1503424, 114), (508438, 114))

In [4]:
if dense_text is False:
    train_text = pickle.load(open('train_text_sparse_new', 'rb'))
    test_text = pickle.load(open('test_text_sparse_new', 'rb'))
else:
    train_text = pickle.load(open('train_text_dense_new', 'rb'))
    test_text = pickle.load(open('test_text_dense_new', 'rb'))
    
train_text.shape, test_text.shape

((1503424, 150), (508438, 150))

In [5]:
if dense_text is False:
    x_test = scipy.sparse.hstack([
        test_text,
        test
    ], format='csr')

    x_train = scipy.sparse.hstack([
        train_text,
        train
    ], format='csr')
else:
    x_test = np.hstack([test_text, test])
    x_train = np.hstack([train_text, train])

In [6]:
text_feature_names = ['text_'+str(i) for i in range(train_text.shape[1])]

In [7]:
feature_names = np.hstack([
    text_feature_names,
    train.columns.tolist(),
])
categorical = pickle.load(open('selected_cat_feature_names', 'rb'))
print('Number of features:', len(feature_names))

Number of features: 264


In [8]:
del train, test, train_text, test_text; gc.collect()

0

In [9]:
target = 'deal_probability'
y_train = pd.read_csv('data/train.csv', usecols=['activation_date', 'deal_probability'], parse_dates=['activation_date']) \
            .sort_values('activation_date') \
            .reset_index(drop=True).deal_probability.values

In [10]:
file_prefix = 'kh_sparse_' if dense_text is False else 'kh_dense_'

pickle.dump(x_train, open(file_prefix+'train_features.pickle', 'wb'))    
pickle.dump(x_test, open(file_prefix+'test_features.pickle', 'wb')) 
pickle.dump(y_train, open(file_prefix+'train_answers.pickle', 'wb'))
pickle.dump(feature_names, open(file_prefix+'feature_names.pickle', 'wb'))  
pickle.dump(categorical, open(file_prefix+'categorical_feature_names', 'wb'))